# Imports

In [14]:
import numpy as np
import pandas as pd
from scipy.io import arff
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

In [15]:
def f1_model(model,X_test,y_test):
    y_pred = model.predict(X_test)
    result = f1_score(y_test,y_pred,pos_label=1,zero_division=0)
    return result

## Reading data

In [16]:
if __name__ == '__main__':
    # Change filename here
    filename = '../datasets/NASA/PC2.arff.txt'

    data = arff.loadarff(filename)
    loaddata = pd.DataFrame(data[0])
    loaddata.head()

## Acquire X and Y values

In [17]:
if __name__ == '__main__':
    # software_metrics = np.array(loaddata[['LOC_BLANK','BRANCH_COUNT','CALL_PAIRS','LOC_CODE_AND_COMMENT']])
    # labels = np.array(loaddata['Defective'])

    software_metrics = np.array(loaddata.iloc[:,:-1]) #Software metrics
    labels = np.array(loaddata.iloc[:,-1]) #Labels

## Train test Split

In [18]:
if __name__ == '__main__':
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(software_metrics, labels, test_size = 0.1)
    y_train = y_train.astype(str)
    y_test = y_test.astype(str)

## Selecting a Model (Logistic Regression)

In [19]:
if __name__ == '__main__':
    from sklearn.linear_model import LogisticRegression

    logreg = LogisticRegression(solver='lbfgs')
    logreg.fit(X_train, y_train)

C:\Users\user\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [20]:
if __name__ == '__main__':    
    y_pred = logreg.predict(X_test)
    y_pred

In [21]:
if __name__ == '__main__':
    pd.DataFrame(confusion_matrix(y_test,y_pred), columns=['Predicted Non-Defective', "Predicted Defective"], index=['Actual Non-Defective', 'Actual Defective'])

In [22]:
if __name__ == '__main__':    
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    print(f'True Positives: {tp}')
    print(f'False Positives: {fp}')
    print(f'True Negatives: {tn}')
    print(f'False Negatives: {fn}')

True Positives: 0
False Positives: 0
True Negatives: 71
False Negatives: 2


## Calculating Precision (True Positive)/(True Positive + False Positive)

In [23]:
if __name__ == '__main__':
    precision = (tp)/(tp+fp)
    print(f'Precision: {precision}')

Precision: nan


<ipython-input-23-f900fed89c97>:2: RuntimeWarning: invalid value encountered in longlong_scalars
  precision = (tp)/(tp+fp)


## Calculating Recall (True Positve)/(True Positive + False Negative)

In [24]:
if __name__ == '__main__':  
    recall = (tp)/(tp+fn)
    print(f'Recall: {recall}')

Recall: 0.0


## Calculating F1-score

In [25]:
if __name__ == '__main__':   
    result = (2*precision*recall)/(precision+recall)
    print(f'F1-score from computation: {result}')

    result = f1_score(y_test,y_pred,pos_label='Y',zero_division=0)
    print(f'F1-score from Import: {result}')
    

F1-score from computation: nan
F1-score from Import: 0.0
